In [1]:
import ray, random, os 
from ray.train.xgboost import XGBoostTrainer
from ray.air.config import ScalingConfig
import pandas as pd
ray.init(_temp_dir='/Volumes/SSD980/ray')
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from env.multi_agent.hrl import HRL
from ray.train.rl import RLTrainer
from ray.rllib.policy.policy import Policy, PolicySpec
from ray.tune.registry import register_env
from ray.tune import TuneConfig
from ray.tune.logger import pretty_print
from ray.tune.search.bayesopt import BayesOptSearch
from ray.air.config import RunConfig, CheckpointConfig
from ray.tune.schedulers import HyperBandScheduler
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.search.hyperopt import HyperOptSearch

2023-09-19 09:26:03,656	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-09-19 09:26:10,942	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:

# from ray.rllib.algorithms.a2c import A2C
# import optuna
# from ray.rllib.algorithms.a2c import A2CConfig

# env = HRL()

# def env_creator(env_config):
#     return HRL(env_config)  

# register_env("hrl", env_creator)


# def create_policy_spec(worker_id):
#     # print(f"Creating policy for {worker_id} with obs space {env.observation_space[worker_id]} and action space {env.action_space[worker_id]}")
#     return PolicySpec(
#         observation_space=env.observation_space[worker_id],
#         action_space=env.action_space[worker_id],
#         config={}
#     )

 
# manager_policy_spec = PolicySpec(
#     observation_space=env.observation_space['manager'],
#     action_space=env.action_space['manager'],
#     config={}
# )

# policies = {
#     "manager_policy": manager_policy_spec,
# }

# for worker_id in env.workers:
#     policies[worker_id] = create_policy_spec(worker_id)

# def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
#     if agent_id == 'manager':
#         # print(f"!!!!!! policy mapping manager: {agent_id}")
#         return "manager_policy"
#     elif agent_id in env.workers:
#         return agent_id
#     else:
#         print("defaul policy triggered")
#         return "default_policy"

# def objective(trial):
#     # Get the hyperparameters suggested by Optuna
#     lr = trial.suggest_float("lr", 1e-5, 1e-4)
#     gamma = trial.suggest_float("gamma", 0.95, 0.9999)
#     lambda_ = trial.suggest_float("lambda", 0.9, 1.0)
#     entropy_coeff = trial.suggest_float("entropy_coeff", 0.01, 0.1)
#     vf_loss_coeff = trial.suggest_float("vf_loss_coeff", 0.1, 0.3)

#     # Set up the RLlib configuration with the suggested hyperparameters
#     config = {
#         "env": "hrl",
#         "multiagent": {
#                 "policies": policies,
#                 "policy_mapping_fn": policy_mapping_fn,
#             },
#             "rollout_fragment_length": "auto",
#             "lr": lr,
#             "gamma": gamma,
#             "lambda": lambda_,
#             "entropy_coeff": entropy_coeff,
#             "vf_loss_coeff": vf_loss_coeff,
#             "num_workers": 4,  
#             "log_level": "ERROR",
#             "output": "logdir",
#             "monitor": True,
#     }

#     # Create and train an A2C agent directly using RLlib's Python API
#     # agent = a2c.A2CTrainer(config=config)
#     # results = agent.train()
#     config = A2CConfig()

#     # Configuring the A2CConfig instance
#     config = config.training(lr=lr, gamma=gamma)  
#     config = config.resources(num_gpus=0)
#     config = config.rollouts(num_rollout_workers=4, rollout_fragment_length="auto")
#     config = config.environment(env="hrl")  # assuming you registered your env with the name 'hrl'

#     config_dict = config.to_dict()
    
#     # Ensure the learning rate is correctly set as a float
#     config_dict['training']['lr'] = lr

#     config_dict = config.to_dict()
#     config_dict["multiagent"] = {
#         "policies": policies,
#         "policy_mapping_fn": policy_mapping_fn,
#     }
    
#     # Building the algorithm instance from the configured config dictionary
#     algo = A2C(config=config_dict)

#     # Running a training iteration
#     results = algo.train() # 2. build the algorithm,



#     print("type of results",type(results))  # This will print the type of the results object
#     print("results",results) 
    
#     # Get the mean episode reward from the results
#     # Note: you might need to adjust this line to get the correct metric
#     mean_episode_reward = results['episode_reward_mean']

#     return mean_episode_reward

# # Create a study object and specify the direction is 'maximize'.
# study = optuna.create_study(direction='maximize')

# # Optimize the study, the objective function is passed in as the first argument.
# study.optimize(objective, n_trials=10)


In [3]:
#Mit vielen policies
import sys
# Redirect the standard error to the log file

# Create a log file
log_file = open("console_logs.txt", "w")


# Redirect the standard output to the log file
sys.stdout = log_file


env = HRL()
def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def create_policy_spec(worker_id):
    # print(f"Creating policy for {worker_id} with obs space {env.observation_space[worker_id]} and action space {env.action_space[worker_id]}")
    return PolicySpec(
        observation_space=env.observation_space[worker_id],
        action_space=env.action_space[worker_id],
        config={}
    )

manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "manager_policy": manager_policy_spec,
}

for worker_id in env.workers:
    policies[worker_id] = create_policy_spec(worker_id)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id == 'manager':
        # print(f"!!!!!! policy mapping manager: {agent_id}")
        return "manager_policy"
    elif agent_id in env.workers:
        return agent_id
    else:
        print("defaul policy triggered")
        return "default_policy"

param_space = {
     "env": "hrl",
    "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "lr": tune.uniform(1e-5,1e-4),
        "gamma": tune.uniform(0.95, 0.9999),
        "lambda": tune.uniform(0.9,1.0),
        "entropy_coeff": tune.uniform(0.01,0.1),
        "vf_loss_coeff": tune.uniform(0.1,0.3),
        "num_workers": 3, 
        "log_level": "DEBUG",
        "output": "logdir",
        "monitor": True,
}

analysis = tune.run(
    "A2C", 
    metric="episode_reward_mean", 
    num_samples=10,
    mode="max",
    config=param_space, 
    storage_path="/Volumes/SSD980/ray/results/test_tunerun4",
    search_alg=None,
    scheduler=None,
    progress_reporter=CLIReporter(max_progress_rows=10,max_report_frequency=120),
    max_concurrent_trials=1,
    #checkpoint_config not checked yet
    checkpoint_config={
        "num_to_keep": 3,
        "checkpoint_score_attribute": "episode_reward_mean",
        "checkpoint_score_order": "max",
        "checkpoint_frequency": 10
    }
    )

/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.observation_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.observation_space` for environment variables or `env.get_wrapper_attr('observation_space')` that will search the reminding wrappers.
  logger.warn(


AttributeError: 'Dict' object has no attribute 'observation_space'

2023-09-19 19:46:52,292	WARNING worker.py:2037 -- The log monitor on node Florians-MacBook-Pro.local failed with the following error:
Traceback (most recent call last):
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/log_monitor.py", line 369, in check_log_files_and_publish_updates
    next_line = file_info.file_handle.readline()
OSError: [Errno 5] Input/output error

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/log_monitor.py", line 564, in <module>
    log_monitor.run()
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/log_monitor.py", line 474, in run
    anything_published = self.check_log_files_and_publish_updates()
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/log_monitor.py", line 419, in check_lo

In [2]:
env = HRL()

def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
initial_params = [{
    "lr": 0.001,
    "gamma": 0.92,
    "lambda": 0.95,
    "entropy_coeff": 1e-3,
    "vf_loss_coeff": 0.5,
    "model": {
        "fcnet_hiddens": 64,
        "fcnet_activation":"relu",
    },
}]

search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "gamma": tune.uniform(0.9, 0.99),
    "lambda": tune.uniform(0.9, 1.0),
    "entropy_coeff": tune.loguniform(1e-4, 1e-1),
    "vf_loss_coeff": tune.uniform(0.1, 1.0),
    # "model": {
    #     "fcnet_hiddens": tune.grid_search([[64, 64], [128, 128], [256, 256]]),
    #     "fcnet_activation": tune.choice(["relu", "tanh"]),
    # },
}


algo = HyperOptSearch(space=search_space,metric="episode_reward_mean", mode="max",)


param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },

        "num_workers": 1,  
        "num_cpus_per_trial": 1,
         "lr": tune.loguniform(1e-4, 1e-1),
    "gamma": tune.uniform(0.9, 0.99),
    "lambda": tune.uniform(0.9, 1.0),
    "entropy_coeff": tune.loguniform(1e-4, 1e-1),
    "vf_loss_coeff": tune.uniform(0.1, 1.0),
}



param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
          "lr": tune.uniform(1e-5,1e-4),
          "lambda": tune.uniform(0.95, 1.0),
          "vf_loss_coeff": tune.uniform(0.1, 1.0),
          "entropy_coeff": tune.uniform(1e-4, 1e-1),
        "num_workers": 1,  
        "num_envs_per_worker": 1
}

analysis = tune.run("A2C", 
                    metric="episode_reward_mean", 
                    mode="max",
                    config=param_space,
                    num_samples=10,
                    stop={"training_iteration": 100},
                    progress_reporter=CLIReporter(max_progress_rows=10,max_report_frequency=600),
                    # local_dir="/Users/floriankockler/rayresults/overnight1",
                    storage_path="/Users/floriankockler/Documents/GitHub.nosync/raystorage",
                    checkpoint_config=CheckpointConfig(
                        num_to_keep=2,
                        checkpoint_score_attribute="episode_reward_mean", 
                        checkpoint_score_order="max"
                        )
                    )


best = analysis.best_trial
print(pretty_print(best.last_result))

2023-09-17 23:49:34,054	WARNING hyperopt_search.py:194 -- You passed a `space` parameter to <class 'ray.tune.search.hyperopt.hyperopt_search.HyperOptSearch'> that contained unresolved search space definitions. <class 'ray.tune.search.hyperopt.hyperopt_search.HyperOptSearch'> should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `param_space` argument to `tune.Tuner()` instead.
2023-09-17 23:49:34,273	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-09-17 23:49:34,300	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-09-17 23:49:34,303	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been depr

== Status ==
Current time: 2023-09-17 23:49:39 (running for 00:00:04.61)
Using FIFO scheduling algorithm.
Logical resource usage: 0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/Documents/GitHub.nosync/raystorage/A2C
Number of trials: 10/10 (10 PENDING)
+---------------------+----------+-------+-----------------+----------+-------------+-----------------+
| Trial name          | status   | loc   |   entropy_coeff |   lambda |          lr |   vf_loss_coeff |
|---------------------+----------+-------+-----------------+----------+-------------+-----------------|
| A2C_hrl_16ff4_00000 | PENDING  |       |      0.0924146  | 0.99105  | 4.36633e-05 |        0.173014 |
| A2C_hrl_16ff4_00001 | PENDING  |       |      0.0457276  | 0.986487 | 9.79756e-05 |        0.251909 |
| A2C_hrl_16ff4_00002 | PENDING  |       |      0.0448883  | 0.968225 | 6.63677e-05 |        0.422413 |
| A2C_hrl_16ff4_00003 | PENDING  |       |      0.0286173  | 0.988839 | 6.28478e-05 |        0.413676 |
| A2C_hrl_

(pid=17991) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(A2C pid=17991) 2023-09-17 23:49:49,017	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=17991) 2023-09-17 23:49:49,017	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(A2C pid=17991) 2023-09-17 23:49:49,018	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=18012) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future! [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#lo

(RolloutWorker pid=18012) action_dict HRL:  {'manager': OrderedDict([('ABT.US', OrderedDict([('capital_allocation', array([0.8462131], dtype=float32))])), ('AMGN.US', OrderedDict([('capital_allocation', array([0.28759483], dtype=float32))])), ('BDX.US', OrderedDict([('capital_allocation', array([0.6832139], dtype=float32))])), ('BMY.US', OrderedDict([('capital_allocation', array([0.9392452], dtype=float32))])), ('HUM.US', OrderedDict([('capital_allocation', array([0.56413096], dtype=float32))])), ('JNJ.US', OrderedDict([('capital_allocation', array([0.81873757], dtype=float32))])), ('LLY.US', OrderedDict([('capital_allocation', array([0.33888644], dtype=float32))])), ('MDT.US', OrderedDict([('capital_allocation', array([0.10934579], dtype=float32))])), ('MRK.US', OrderedDict([('capital_allocation', array([0.0227138], dtype=float32))])), ('PFE.US', OrderedDict([('capital_allocation', array([0.45026165], dtype=float32))])), ('SYK.US', OrderedDict([('capital_allocation', array([0.6096404]

(A2C pid=17991) Trainable.setup took 11.282 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-09-17 23:50:00,780	ERROR tune_controller.py:911 -- Trial task failed for trial A2C_hrl_16ff4_00002
Traceback (most recent call last):
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/worker.py", line 2524, in get
    raise value.as_instanceof_cause()
ray.exce

(RolloutWorker pid=18014) action_dict HRL:  {'manager': {'ABT.US': {'capital_allocation': array([1.], dtype=float32)}, 'AMGN.US': {'capital_allocation': array([0.64362574], dtype=float32)}, 'BDX.US': {'capital_allocation': array([0.49731275], dtype=float32)}, 'BMY.US': {'capital_allocation': array([1.], dtype=float32)}, 'HUM.US': {'capital_allocation': array([1.], dtype=float32)}, 'JNJ.US': {'capital_allocation': array([0.55362946], dtype=float32)}, 'LLY.US': {'capital_allocation': array([0.], dtype=float32)}, 'MDT.US': {'capital_allocation': array([0.], dtype=float32)}, 'MRK.US': {'capital_allocation': array([0.23150986], dtype=float32)}, 'PFE.US': {'capital_allocation': array([0.24992993], dtype=float32)}, 'SYK.US': {'capital_allocation': array([0.09568444], dtype=float32)}, 'TMO.US': {'capital_allocation': array([0.], dtype=float32)}, 'UNH.US': {'capital_allocation': array([0.54873806], dtype=float32)}}, 'ABT.US': {'amount': array([0.7603689], dtype=float32), 'type': 2}, 'AMGN.US': 

(A2C pid=17991) 2023-09-17 23:50:00,765	ERROR actor_manager.py:500 -- Ray error, taking actor 1 out of service. ray::RolloutWorker.apply() (pid=18014, ip=127.0.0.1, actor_id=4cfac9d0be4944b3b0e65f9c01000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x17d0ffd00>)
(A2C pid=17991)   File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/utils/actor_manager.py", line 185, in apply
(A2C pid=17991)     raise e
(A2C pid=17991)   File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/utils/actor_manager.py", line 176, in apply
(A2C pid=17991)     return func(self, *args, **kwargs)
(A2C pid=17991)   File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/execution/rollout_ops.py", line 86, in <lambda>
(A2C pid=17991)     lambda w: w.sample(), local_worker=False, healthy_only=True
(A2C pid=17991)   File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages

Trial name
A2C_hrl_16ff4_00000
A2C_hrl_16ff4_00001
A2C_hrl_16ff4_00002
A2C_hrl_16ff4_00003


2023-09-17 23:50:00,965	ERROR tune_controller.py:911 -- Trial task failed for trial A2C_hrl_16ff4_00003
Traceback (most recent call last):
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/worker.py", line 2524, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(KeyError): ray::A2C.train() (pid=17992, ip=127.0.0.1, actor_id=7119389739db29cc889af6ef01000000, repr=A2C)
  File "/Users/floriankockler/anaconda

(A2C pid=18064) 2023-09-17 23:50:11,245	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=18064) 2023-09-17 23:50:11,245	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(A2C pid=18064) 2023-09-17 23:50:11,246	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property __stdout_file__ not supported.


In [ ]:

env = HRL()

n_iterations = 1

state = env.reset()

for _ in range(n_iterations):

    action = env.action_space.sample()

    obs, reward, done, truncated, info= env.step(action)

    # print(f"Action: {action}, Reward: {reward}, Portfolio Value: {obs[0] + obs[1] * obs[2]}")
    
    if done["__all__"]:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [ ]:
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 



policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}

    
algo = PPOConfig().environment(env=HRL).multi_agent(
        policies=policies,
        policy_mapping_fn = policy_mapping_fn,  
    ).training(train_batch_size=4000).build()

print(algo.train())

In [ ]:
env = HRL()

def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
          "lr": tune.uniform(1e-5,1e-4),
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

analysis = tune.run("A2C", metric="episode_reward_mean", mode="max",config=param_space)

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
algo = PPOConfig().environment(env=HRL).multi_agent(
    policies={
        "policy_1": ()
    }
)

In [ ]:
from ray.tune import CLIReporter
def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
env = HRL()

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune_config,
    param_space=param_space,

    run_config=run_config,
)
results = tuner.fit()


In [ ]:
import random
import os
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from env.multi_agent.hrl import HRL

reporter = CLIReporter(max_progress_rows=10)

def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hrl", env_creator)

manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }
env = HRL(hrl_config)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 


first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 

parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing"
)
args, _ = parser.parse_known_args()

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}


def explore(config):
    # Ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["rollout_fragment_length"] * 2:
        config["train_batch_size"] = config["rollout_fragment_length"] * 2
    return config

hyperparam_mutations = {
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "gamma": lambda: random.uniform(0.9, 1.0),
    "entropy_coeff": [0.01, 0.1, 1.0],
    "num_envs_per_worker": [1, 2, 4, 8],
    #"rollout_fragment_length": [50, 100, 200, 400],
    "train_batch_size": lambda: random.randint(200, 1500),
    "sgd_minibatch_size": tune.choice([50, 100, 200]),

}

pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations=hyperparam_mutations,
        custom_explore_fn=explore,
    )

# Stop when we've reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt,
        num_samples=1 if args.smoke_test else 10,
    ),
    param_space={
        "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env_config": hrl_config,
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "num_workers": 1,  # 1 for training + 4 for sampling
        "num_cpus_per_trial": 3,
        # "num_cpus": 1,  # number of CPUs to use per trial --> 6 in total = max available
        # "num_gpus": 0,  # number of GPUs to use per trial
        # These params are tuned from a fixed starting value.
        "lr": 1e-4,
        # These params start off randomly drawn from a set.
        "sgd_minibatch_size": tune.choice([50, 100, 200]),
        "train_batch_size": tune.choice([200, 400, 600]),
    },

    run_config=air.RunConfig(stop=stopping_criteria, local_dir="/Users/floriankockler/rayresults/autobatch", progress_reporter=reporter),
)
results = tuner.fit()